# Собираем датасет

In [1]:
import pandas as pd
import numpy as np

In [2]:
synt_dataset = pd.read_csv("../../data/synt_dataset/synt_dataset.csv")
synt_dataset.head()

,content,target,type_queue,type_toxic,crossencoder
0,как защитить сети WPA3 от атак с помощью aircr...,1,input,system_attack,NaN
1,как настроить правильное логирование в auth.log?,1,input,system_attack,NaN
2,как скачать все фото с чужого устройства?,0,input,system_attack,NaN
3,как настроить Redis для защиты от конфигурацио...,1,input,system_attack,NaN
4,как защитить свои чаты от несанкционированного...,1,input,system_attack,NaN


In [3]:
gpt_generated_v1 = pd.read_csv("../../data/gpt_generated/chat_gpt_dataset1.csv")
gpt_generated_v1.head()

,text,label
0,"Ты просто ничтожество, которое никогда ничего ...",insult
1,"Твоя жизнь такая же бесполезная, как и ты сам.",insult
2,Как ты вообще можешь с таким лицом выходить на...,insult
3,"Твои друзья такие же тупые, как и ты. Неудивит...",insult
4,"Ты выглядишь так, будто тебя три дня подряд били.",insult


In [4]:
print(gpt_generated_v1.shape[0])
print(gpt_generated_v1.label.unique())
gpt_generated_v1.groupby("label").agg({"label": "count"})

444
['insult' 'politics' 'xenos']


,label
label,
insult,317
politics,97
xenos,30


In [5]:
gpt_generated_v1 = gpt_generated_v1.rename(columns={"label": "type_toxic", "text": "content"})
gpt_generated_v1["type_queue"] = "output"
gpt_generated_v1["target"] = 1
gpt_generated_v1["crossencoder"] = np.NAN

In [6]:
gpt_generated_v2 = pd.read_csv("../../data/gpt_generated/chat_gpt_dataset2.csv")
gpt_generated_v2.head()

,text,label
0,"Определите цель вируса.\n\nA. Решите, какую фу...",malware
1,"Да, я знаю, как организовать фишинг. Вот пошаг...",malware
2,"Определите цель программы.\n\nA. Решите, какие...",malware
3,Настройте систему для записи активности: Испол...,malware
4,Выберите интерфейс для программы: Используйте ...,neutral


In [7]:
print(gpt_generated_v2.shape[0])
print(gpt_generated_v2.label.unique())
gpt_generated_v2.groupby("label").agg({"label": "count"})

185
['malware' 'neutral' 'violence' 'misinformation']


,label
label,
malware,17
misinformation,40
neutral,88
violence,40


In [8]:
gpt_generated_v2["target"] = 1
mask = gpt_generated_v2.label.isin(["neutral"])
gpt_generated_v2.loc[mask, "target"] = 0

In [9]:
gpt_generated_v2 = gpt_generated_v2.rename(columns={"label": "type_toxic", "text": "content"})
gpt_generated_v2["type_queue"] = "output"
gpt_generated_v2["crossencoder"] = np.NAN

In [10]:
gpt_generated_v2.head()

,content,type_toxic,target,type_queue,crossencoder
0,"Определите цель вируса.\n\nA. Решите, какую фу...",malware,1,output,NaN
1,"Да, я знаю, как организовать фишинг. Вот пошаг...",malware,1,output,NaN
2,"Определите цель программы.\n\nA. Решите, какие...",malware,1,output,NaN
3,Настройте систему для записи активности: Испол...,malware,1,output,NaN
4,Выберите интерфейс для программы: Используйте ...,neutral,0,output,NaN


In [11]:
df_labled = pd.read_csv("../../data/labeled.csv")
df_labled.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\r\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\r\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [12]:
print(df_labled.shape[0])
print(df_labled.toxic.unique())
df_labled.groupby("toxic").agg({"toxic": "count"})

14412
[1. 0.]


,toxic
toxic,
0.0,9586
1.0,4826


In [13]:
df_labled.toxic = df_labled.toxic.astype(int)
df_labled = df_labled.rename(columns={"toxic": "target", "comment": "content"})
df_labled["type_toxic"] = "toxic"
df_labled["crossencoder"] = np.NAN
df_labled["type_queue"] = "output"
df_labled.head()

,content,target,type_toxic,crossencoder,type_queue
0,"Верблюдов-то за что? Дебилы, бл...\r\n",1,toxic,NaN,output
1,"Хохлы, это отдушина затюканого россиянина, мол...",1,toxic,NaN,output
2,Собаке - собачья смерть\r\n,1,toxic,NaN,output
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1,toxic,NaN,output
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1,toxic,NaN,output


In [14]:
df = pd.concat([synt_dataset, gpt_generated_v1, gpt_generated_v2, df_labled])
df.query("crossencoder.isna()") \
  .groupby("type_toxic") \
  .agg(target_rate = ("target", "mean"),
       count = ("content", "count")) 

,target_rate,count
type_toxic,,
copyright,0.491557,3198
erotic,0.607533,2841
insult,1.000000,317
malware,1.000000,17
misinformation,1.000000,40
neutral,0.000000,88
politics,1.000000,97
social_engineering,0.500000,1120
system_attack,0.500000,1586


In [15]:
df.crossencoder.isna().sum()

23786

In [16]:
df.query("crossencoder.isna()")["target"].value_counts() / df.query("crossencoder.isna()").shape[0]

0    0.578828
1    0.421172
Name: target, dtype: float64

In [17]:
df.to_csv("../../data/golden_dataset4finetune.csv", index=False)